In [1]:
import bs4
import requests
from urllib.parse import urlparse, parse_qs
import pandas as pd
import unicodedata
import re

In [2]:
urls = []
names = []
ids = []

for page_number in range(1, 64):
    URL = f"https://www.sadevillam.com/kiralik-villalar?s={page_number}"
    response = requests.get(URL)
    response.encoding = "utf-8"  
    html_parsed = bs4.BeautifulSoup(response.text, "html.parser")
    
    villa_url_html = html_parsed.select("div.content  div#searchResult div.resultItem a")
    for index in range(0, len(villa_url_html), 2):
        villa_url = "https://www.sadevillam.com" + villa_url_html[index]["href"]
        urls.append(villa_url)
        
    for name in html_parsed.select("div.resultItem strong.title"):
        villa_name = name.getText().strip()
        villa_id = villa_name.lower().replace(" ", "")
        names.append(villa_name)
        ids.append(villa_id)
        
    if page_number % 10 == 0:
        print(f"{page_number} pages have been check!")

print(f"\n---OPERATION HAS BEEN COMPLETED!---")
print(f"{page_number} pages have been checked!")
print(f"Total Villa Name: {len(names)}")
print(f"Total Villa URL: {len(urls)}")

10 pages have been check!
20 pages have been check!
30 pages have been check!
40 pages have been check!
50 pages have been check!
60 pages have been check!

---OPERATION HAS BEEN COMPLETED!---
63 pages have been checked!
Total Villa Name: 1117
Total Villa URL: 1117


In [3]:
addresses = []
latitudes = []
longitudes = []
photos = []
average_prices = []
q1_prices = []
q2_prices = []
q3_prices = []
q4_prices = []
bedrooms = []
guests = []
bathrooms = []
descriptions = []

URL_ERRORS = []
villa_number = 0
for URL in urls:
    try:
        response = requests.get(URL)
        response.encoding = "utf-8" 
        html_parsed = bs4.BeautifulSoup(response.text, "html.parser")
        villa_number += 1
    except:
        print(f"{URL} : URL ERROR!")
        URL_ERRORS.append(URL)
        
    try:
        address = html_parsed.find("span", class_="location").text.strip()        
    except:
        address = None
        print(f"{URL} : Address is not found!")
        
    try:
        location_url = unicodedata.normalize("NFKD", str(html_parsed.select_one(".distanceLocation")["href"]))
        pattern = r'll=([0-9]+\.[0-9]+),([0-9]+\.[0-9]+)'
        match = re.search(pattern, location_url)
        latitude = float(match.group(2))
        longitude = float(match.group(1))
    except:
        latitude = None
        longitude = None
        print(f"{URL} : Location is not found!")
        
    try:
        photos_html = html_parsed.select_one("div#body div#mp-pusher div.main div.wrapper div.content div.fotorama img")
        base = "https://www.sadevillam.com"
        photo = tuple(base + image_source["src"] for image_source in photos_html.find_all("img")[0]) 
    except:
        print(f"Image Error {URL}")
        photo = None
    
    try:
        total_price = 0
        villa_prices = [villa_price.getText().replace(".", "").strip() for villa_price in html_parsed.select("div.row ul.weeklyPricing li span")]
        for villa_price in villa_prices:
            if "TL" in villa_price or "₺" in villa_price:
                total_price += int(villa_price.replace("TL", "").replace("₺", "").strip()) // 7
            elif "£" in villa_price:
                total_price += int(villa_price.replace("£", "").strip()) * 35 // 7
        
        avg_price = total_price // len(villa_prices)
        q1_price = avg_price*4 * 0.14
        q2_price = avg_price*4 * 0.32
        q3_price = avg_price*4 * 0.37
        q4_price = avg_price*4 * 0.17

    except:
        print(f"{URL} : PRICE ERROR!")
        avg_price = None
        q1_price = None
        q2_price = None
        q3_price = None
        q4_price = None
        
    try:
        guest = int(html_parsed.select(".detailProp span")[0].getText()[0])
    except:
        print(f"{URL} : GUEST NUMBER ERROR!")
        guests.append(guest)
        
    try:
        bedroom = int(html_parsed.select(".detailProp span")[1].getText()[0])
    except:
        print(f"{URL} : BEDROOM NUMBER ERROR!")
        bedrooms.append(bedroom)
        
    try:
        bathroom = int(html_parsed.select(".detailProp span")[2].getText()[0])
    except:
        print(f"{URL} : BATHROOM NUMBER ERROR!")
        bathrooms.append(bathroom)
        
    try:
        if len(html_parsed.select(".content p span span")) != 0:
            description = (unicodedata.normalize("NFKD", "\n".join([desc.getText().strip() for desc in html_parsed.select(".content p span span")[:3]]))).strip()
        else:
            description = (unicodedata.normalize("NFKD", "\n".join([desc.getText().strip() for desc in html_parsed.select(".content p")]))).strip()
    
    except:
        print(f"{URL} : DESCRIPTION ERROR!")
        description = None
        
    latitudes.append(latitude)
    longitudes.append(longitude)
    addresses.append(address) 
    photos.append(photo)
    average_prices.append(avg_price)
    q1_prices.append(q1_price)
    q2_prices.append(q2_price)
    q3_prices.append(q3_price)
    q4_prices.append(q4_price)
    guests.append(guest)
    bedrooms.append(bedroom)
    bathrooms.append(bathroom)
    descriptions.append(description)
    
    if villa_number % 75 == 0:
        print(f"{villa_number} villas have been completed!")

print(f"THE OPERATION IS COMPLETED!")
print(f"TOTAL VILLA NUMBER: {villa_number}")       

https://www.sadevillam.com/Villa-Kivi : Location is not found!
https://www.sadevillam.com/Villa-Sazak-Duo : Location is not found!
https://www.sadevillam.com/Villa-Simay : Location is not found!
https://www.sadevillam.com/Villa-Begon-2 : Location is not found!
https://www.sadevillam.com/Villa-Cosy : Location is not found!
https://www.sadevillam.com/Villa-Yildiz : Location is not found!
https://www.sadevillam.com/Villa-Berlin : Location is not found!
https://www.sadevillam.com/Villa-Sirius- : PRICE ERROR!
https://www.sadevillam.com/Villa-Mirbis : Location is not found!
https://www.sadevillam.com/Villa-Mevsim : Location is not found!
https://www.sadevillam.com/Villa-Nostalji : Location is not found!
https://www.sadevillam.com/Villa-Munih : Location is not found!
https://www.sadevillam.com/Villa-Kirca- : Location is not found!
https://www.sadevillam.com/Villa-Neon : Location is not found!
https://www.sadevillam.com/Villa-Amore-Garden : Location is not found!
https://www.sadevillam.com/Vil

https://www.sadevillam.com/Villa-Lavinya : Location is not found!
https://www.sadevillam.com/Villa-Sedef : Location is not found!
https://www.sadevillam.com/Villa-Lebon : Location is not found!
https://www.sadevillam.com/Villa-Huzur : Location is not found!
https://www.sadevillam.com/Villa-Alya-ikizce : Location is not found!
https://www.sadevillam.com/Villa-Kelebek-1 : Location is not found!
https://www.sadevillam.com/Villa-Kelebek-2 : Location is not found!
https://www.sadevillam.com/Villa-Emir-1 : Location is not found!
https://www.sadevillam.com/Villa-Emir-2 : Location is not found!
https://www.sadevillam.com/Villa-Buglem : Location is not found!
https://www.sadevillam.com/Villa-Vadi-Exclusive : Location is not found!
https://www.sadevillam.com/Villa-Rimel : Location is not found!
https://www.sadevillam.com/Villa-Minik-Korsan- : Location is not found!
https://www.sadevillam.com/Villa-Rose : Location is not found!
https://www.sadevillam.com/Villa-Hemera : Location is not found!
http

https://www.sadevillam.com/Villa-Lilith : Location is not found!
https://www.sadevillam.com/Villa-Dionysos : Location is not found!
https://www.sadevillam.com/Villa-Koln : Location is not found!
https://www.sadevillam.com/Villa-Goreme- : Location is not found!
https://www.sadevillam.com/Villa-sirinyer : Location is not found!
https://www.sadevillam.com/Villa-Ecem-Su : Location is not found!
https://www.sadevillam.com/Villa-Night : PRICE ERROR!
https://www.sadevillam.com/Villa-Armada : Location is not found!
https://www.sadevillam.com/Villa-Ayaz-1 : Location is not found!
https://www.sadevillam.com/Villa-Ayaz-2 : Location is not found!
https://www.sadevillam.com/Villa-Paix : Location is not found!
https://www.sadevillam.com/Villa-Querencia : Location is not found!
https://www.sadevillam.com/Villa-Blue-Sky : PRICE ERROR!
https://www.sadevillam.com/Villa-Perseid : Location is not found!
825 villas have been completed!
https://www.sadevillam.com/Villa-Batik-1 : Location is not found!
https

In [4]:
data_raw = {
    "ID": ids,
    "Name": names,
    "Address": addresses,
    "Guest Number": guests,
    "Bedroom Number": bedrooms,
    "Bathroom Number":bathrooms,
    "Latitude": latitudes,
    "Longitude": longitudes,
    "Description": descriptions,
    "Photo URLs": photos,
    "Average Price": average_prices,
    "Q1 Price": q1_prices,
    "Q2 Price": q2_prices,
    "Q3 Price": q3_prices,
    "Q4 Price": q4_prices
}

df_raw = pd.DataFrame(data_raw)
df_raw.set_index("ID", inplace=True)
df_raw.to_excel("sadevillam_raw.xlsx")

data_info = {
    "ID": ids,
    "Name": names,
    "Address": addresses,
    "Guest Number": guests,
    "Bedroom Number": bedrooms,
    "Bathroom Number": bathrooms,
    "Description": descriptions,
    "Photo URLs": photos,
    "Latitude": latitudes,
    "Longitude": longitudes
}

df_info = pd.DataFrame(data_info)
df_info.set_index("ID", inplace=True)
df_info.to_excel("sadevillam_info.xlsx")

data_financial = {
    "ID": ids,
    "Average Price": average_prices,
    "Q1 Price": q1_prices,
    "Q2 Price": q2_prices,
    "Q3 Price": q3_prices,
    "Q4 Price": q4_prices   
}

df_financial = pd.DataFrame(data_financial)
df_financial.set_index("ID", inplace=True)
df_financial.to_excel("sadevillam_financial.xlsx")